#### Process
- 1. 동이름 > 위도, 경도 수집
- 2. 위도, 경도 > geohash(영역값)
- 3. geohash > 매물아이디
- 4. 매물아이디 > 매물정보

In [1]:
# !pip install geohash2

In [2]:
import requests
import pandas as pd
import geohash2

In [3]:
def oneroom(addr):
    # 1. 동이름 > 위도, 경도
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    # 2. 위도, 경도 > geohash
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # 3. geohash > 매물 아이디
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()['items']
    ids = [item['item_id'] for item in items]
    
    # 4. 매물 아이디 > 매물 정보
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain': 'zigbang',
        'withCoalition': 'true',
        'item_ids': ids[:900]
    }
    response = requests.post(url, params)
    items = response.json()['items']
    columns = ['item_id', 'sales_type', 'deposit', 'rent', 'size_m2', 'address1', 'manage_cost']
    return pd.DataFrame(items)[columns]

In [4]:
addr = '강서구 등촌동'
oneroom(addr)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32789882,전세,22700,0,30.66,서울시 강서구 화곡동,4.5
1,32905926,전세,22900,0,30.66,서울시 강서구 화곡동,4.5
2,32910288,전세,24000,0,36.46,서울시 강서구 화곡동,4.5
3,32887801,전세,22300,0,45.70,서울시 강서구 화곡동,11
4,32761004,전세,22600,0,43.21,서울시 강서구 화곡동,11
...,...,...,...,...,...,...,...
895,32887803,전세,10000,0,33.06,서울시 강서구 화곡동,15
896,32776227,전세,48000,0,57.16,서울시 강서구 화곡동,4
897,32582201,전세,21000,0,33.06,서울시 강서구 화곡동,3.5
898,32615624,전세,21400,0,45.04,서울시 강서구 화곡동,4
